## Segmentação de oleodutos subaquáticos via Detectron 2

A etapa de segmentação do oleoduto é crucial para a identificação precisa de sua posição e extensão nas imagens obtidas pelo ROV. 

Nessa etapa, o objetivo é separar o oleoduto do restante da cena, destacando-o de maneira clara e precisa. Para isso, utilizamos o framework Detectron 2, que é uma poderosa ferramenta de segmentação baseada em redes neurais convolucionais.

#### Instalação das bibliotecas necessárias para o projeto

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'detectron2' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Importação das bibliotecas utilizadas

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
import numpy as np
import pandas as pd
from skimage.draw import polygon2mask
import cv2, os, csv, random, sys
from importlib import reload
import pycocotools

ModuleNotFoundError: No module named 'skimage'

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import config
from detectron2.engine.defaults import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import *
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator

#### Leitura dos arquivos de configuração da rede e dados

In [8]:
actual_path = os.getcwd()
root_path = os.path.dirname(actual_path)

PATH_TO_TRAIN_OUTPUT = f"{root_path}/output/"
PATH_TO_LIB_FILES =  f"{root_path}/libs/"

sys.path.append(PATH_TO_LIB_FILES)

In [ ]:
import train_config
from utils import *

reload(train_config)

train_config.cfg.OUTPUT_DIR = PATH_TO_TRAIN_OUTPUT

In [ ]:
database_header = ['frame_id', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'image_name']

df_database = pd.read_csv(
    f'{root_path}/database/database_annotations.csv',
    sep=';',
    header=None,
    names=database_header,
)

In [ ]:
from sklearn.model_selection import train_test_split

# Divide o dataframe em treino e teste (70% treino, 30% teste). Shuffle = true por padrão da biblioteca
train_df, test_df = train_test_split(df_database, train_size=0.7, shuffle=True, random_state=1)

# Divide o dataframe em treino e validação (95% treino, 5% validação). Shuffle = true por padrão da biblioteca
train_df, val_df = train_test_split(train_df, train_size=0.95, shuffle=True, random_state=1)

In [ ]:
#Aplicando schema no dataframe de train para garantir que todos os valores serão int
#Aqui eu ignoro a primeira e última coluna que são nome e path da imagem.

for column in database_header[1:-1]:
    train_df[column] = train_df[column].astype(int)

for column in database_header[1:-1]:
    val_df[column] = val_df[column].astype(int)

In [ ]:
print("[detectron_train.py] Creating Train DatasetCatalog and loading images...")

DatasetCatalog.register("pipeline_train_dataset", lambda: load_pipeline_dataset(train_df))
MetadataCatalog.get("pipeline_train_dataset").set(thing_classes=["pipeline"])

pipeline_train_metadata = MetadataCatalog.get("pipeline_train_dataset")

print("[detectron_train.py] Creating Val DatasetCatalog and loading images...")

DatasetCatalog.register("pipeline_val_dataset", lambda: load_pipeline_dataset(val_df))
MetadataCatalog.get("pipeline_val_dataset").set(thing_classes=["pipeline"])

pipeline_val_metadata = MetadataCatalog.get("pipeline_val_dataset")

[detectron_train.py] Creating Train DatasetCatalog and loading images...
[detectron_train.py] Creating Val DatasetCatalog and loading images...


In [ ]:
from detectron2.data.datasets import convert_to_coco_json

"""Criando um arquivo JSON no formato COCO para os dados de validação. 
Essa etapa é particularmente útil para podermos rodar a validação durante o treino e computar as métricas de AP."""

convert_to_coco_json("pipeline_val_dataset", "/content/drive/MyDrive/detectron2/runs/experimento3/config1 - (80 20)/inference/pipeline_val_dataset_coco_format.json", False)

#### Treinamento do modelo

Antes de realizarmos o treinamento do modelo, é preciso definirmos nosso método de avaliação do modelo. Usaremos o evaluator do COCO framework para fazermos o 'evaluate' durante o treinamento e após o treinamento (no conjunto de teste).

Para isso, criamos a classe MyTrainer, a partir da classe Default, sobrescrevendo apenas o evaluator e definindo-o como o COCOEvaluator

In [ ]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder, allow_cached_coco=False)

In [ ]:
print("[detectron_train.py] Start training...")

os.makedirs(train_config.cfg.OUTPUT_DIR, exist_ok=True)

trainer = MyTrainer(train_config.cfg)

trainer.resume_or_load(resume=False)
trainer.train()

print("[detectron_train.py] Trained sucessfully...")

[detectron_train.py] Start training...
[06/15 22:54:33 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBat

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[06/15 23:06:05 d2.utils.events]:  eta: 0:08:18  iter: 19  total_loss: 56.24  loss_cls: 13.77  loss_box_reg: 2.561  loss_mask: 2.405  loss_rpn_cls: 15.44  loss_rpn_loc: 16.39    time: 0.2662  last_time: 0.3226  data_time: 0.0219  last_data_time: 0.0172   lr: 3.3397e-05  max_mem: 1179M
[06/15 23:06:19 d2.utils.events]:  eta: 0:09:59  iter: 39  total_loss: 12.87  loss_cls: 2.804  loss_box_reg: 0.6194  loss_mask: 0.7671  loss_rpn_cls: 2.188  loss_rpn_loc: 4.268    time: 0.3928  last_time: 0.4367  data_time: 0.0110  last_data_time: 0.0156   lr: 6.7499e-05  max_mem: 1424M
[06/15 23:06:27 d2.utils.events]:  eta: 0:13:00  iter: 59  total_loss: 6.694  loss_cls: 2.084  loss_box_reg: 0.4901  loss_mask: 0.5774  loss_rpn_cls: 0.9004  loss_rpn_loc: 1.731    time: 0.3995  last_time: 0.4088  data_time: 0.0090  last_data_time: 0.0068   lr: 0.0001016  max_mem: 1426M
[06/15 23:06:35 d2.utils.events]:  eta: 0:13:03  iter: 79  total_loss: 6.212  loss_cls: 1.586  loss_box_reg: 0.4352  loss_mask: 0.5597  lo

#### Limpeza dos Datasets

Abaixo realizamos a limpeza dos Datasets criados. Como estamos em um ambiente interativo (notebook), é interessante limparmos os catálogos para que caso precisemos rodar o treinamento novamente eles sejam recriados.

In [ ]:
#Removing dataset registered

print("Removendo os datasets registrados")

DatasetCatalog.remove("pipeline_train_dataset")
DatasetCatalog.remove("pipeline_val_dataset")

Removendo os datasets registrados
